# Phase II: Data Exploration and Analysis

# 1. Data Cleaning and Preparation

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [51]:
df = pd.read_csv('covidcast_new.csv')
df.head()

,time_value,geo_value,smoothed_wspent_time_1d,smoothed_wtested_14d,smoothed_wpublic_transit_1d,smoothed_wworried_become_ill,smoothed_wvaccine_likely_govt_health,smoothed_wshop_1d,smoothed_wtested_positive_14d,smoothed_wwork_outside_home_1d,smoothed_wothers_masked,smoothed_wcli,smoothed_wcovid_vaccinated,smoothed_wvaccine_likely_friends,smoothed_wrestaurant_1d,smoothed_wvaccine_likely_politicians,smoothed_wvaccine_likely_who,smoothed_wwearing_mask,smoothed_wlarge_event_1d
0,1/7/2021,1000,34.412959,12.541558,1.464798,65.771681,22.086274,52.854667,29.288429,30.487852,57.334785,3.244379,2.6382,28.469582,18.571266,9.482316,24.962112,84.311759,12.627098
1,1/7/2021,1003,32.814153,10.189340,1.270183,61.516584,25.179399,51.160278,NaN,28.384842,53.172285,1.199572,NaN,27.532881,23.406516,8.933682,28.864558,82.778220,11.727249
2,1/7/2021,1015,35.621311,14.591271,6.594649,NaN,NaN,54.503711,NaN,37.003522,NaN,3.890709,NaN,NaN,20.974363,NaN,NaN,NaN,13.255336
3,1/7/2021,1051,40.453615,14.164503,3.320922,68.718538,19.438763,54.948755,NaN,37.637900,67.550815,3.284647,NaN,30.223647,20.009477,7.570250,26.494057,86.587182,13.823318
4,1/7/2021,1069,39.228766,10.103564,0.288897,59.961036,23.065494,60.118290,NaN,36.156326,55.995745,1.881970,NaN,30.806929,16.574756,12.916249,24.747620,85.825453,10.258936


In [52]:
unique_geo_values = df['geo_value'].unique()
print(unique_geo_values)

[ 1000  1003  1015  1051  1069  1073  1081  1083  1089  1095  1097  1101
  1103  1117  1125  2000  2020  2090  2170  4000  4003  4013  4015  4019
  4021  4025  4027  5000  5007  5031  5045  5051  5119  5125  5131  5143
  6000  6001  6007  6013  6017  6019  6023  6025  6029  6031  6033  6037
  6039  6041  6045  6047  6053  6055  6057  6059  6061  6065  6067  6071
  6073  6075  6077  6079  6081  6083  6085  6087  6089  6095  6097  6099
  6107  6111  6113  8000  8001  8005  8013  8031  8035  8041  8045  8059
  8069  8077  8101  8123  9000  9001  9003  9005  9007  9009  9011  9013
  9015 10000 10001 10003 10005 11001 12000 12001 12005 12009 12011 12015
 12017 12019 12021 12031 12033 12035 12053 12055 12057 12061 12069 12071
 12073 12081 12083 12085 12086 12091 12095 12097 12099 12101 12103 12105
 12109 12111 12113 12115 12117 12119 12127 12131 13000 13021 13051 13057
 13063 13067 13073 13077 13089 13115 13117 13121 13135 13139 13151 13153
 13185 13215 13223 13245 15000 15001 15003 15009 16

In [53]:
import pandas as pd

# Load FIPS reference
fips_df = pd.read_excel("US_FIPS_codes.xlsx", dtype=str)

# Ensure FIPS codes in the reference table are correctly padded
fips_df["FIPS State"] = fips_df["FIPS State"].str.zfill(2)
fips_df["FIPS County"] = fips_df["FIPS County"].str.zfill(3)

# Ensure geo_value is a 5-digit zero-padded string
df["geo_value"] = df["geo_value"].astype(str).str.zfill(5)

# Extract FIPS State and FIPS County from geo_value
df["FIPS State"] = df["geo_value"].str[:2]
df["FIPS County"] = df["geo_value"].str[-3:]

# Step 1: Merge with full FIPS (State + County)
merged_df = df.merge(fips_df, on=["FIPS State", "FIPS County"], how="left")

# Step 2: Fallback merge on FIPS State to recover State names
fallback_state = fips_df[["FIPS State", "State"]].drop_duplicates()
merged_df = merged_df.merge(fallback_state, on="FIPS State", how="left", suffixes=('', '_fallback'))

# Use fallback State name where missing
merged_df["State"] = merged_df["State"].fillna(merged_df["State_fallback"])

# Clean up temporary column
merged_df.drop(columns=["State_fallback"], inplace=True)


In [54]:
merged_df.head()



,time_value,geo_value,smoothed_wspent_time_1d,smoothed_wtested_14d,smoothed_wpublic_transit_1d,smoothed_wworried_become_ill,smoothed_wvaccine_likely_govt_health,smoothed_wshop_1d,smoothed_wtested_positive_14d,smoothed_wwork_outside_home_1d,smoothed_wothers_masked,smoothed_wcli,smoothed_wcovid_vaccinated,smoothed_wvaccine_likely_friends,smoothed_wrestaurant_1d,smoothed_wvaccine_likely_politicians,smoothed_wvaccine_likely_who,smoothed_wwearing_mask,smoothed_wlarge_event_1d,FIPS State,FIPS County,State,County Name
0,1/7/2021,01000,34.412959,12.541558,1.464798,65.771681,22.086274,52.854667,29.288429,30.487852,57.334785,3.244379,2.6382,28.469582,18.571266,9.482316,24.962112,84.311759,12.627098,01,000,Alabama,NaN
1,1/7/2021,01003,32.814153,10.189340,1.270183,61.516584,25.179399,51.160278,NaN,28.384842,53.172285,1.199572,NaN,27.532881,23.406516,8.933682,28.864558,82.778220,11.727249,01,003,Alabama,Baldwin
2,1/7/2021,01015,35.621311,14.591271,6.594649,NaN,NaN,54.503711,NaN,37.003522,NaN,3.890709,NaN,NaN,20.974363,NaN,NaN,NaN,13.255336,01,015,Alabama,Calhoun
3,1/7/2021,01051,40.453615,14.164503,3.320922,68.718538,19.438763,54.948755,NaN,37.637900,67.550815,3.284647,NaN,30.223647,20.009477,7.570250,26.494057,86.587182,13.823318,01,051,Alabama,Elmore
4,1/7/2021,01069,39.228766,10.103564,0.288897,59.961036,23.065494,60.118290,NaN,36.156326,55.995745,1.881970,NaN,30.806929,16.574756,12.916249,24.747620,85.825453,10.258936,01,069,Alabama,Houston


In [55]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)



In [56]:
null_state_rows = merged_df[merged_df["State"].isnull()]
print(null_state_rows[["FIPS State", "FIPS County", "geo_value"]])  # or add other columns as needed



Empty DataFrame
Columns: [FIPS State, FIPS County, geo_value]
Index: []


In [57]:
null_count = df["smoothed_wtested_positive_14d"].isnull().sum()
print(f"Number of rows with null 'smoothed_wtested_positive_14d': {null_count}")


Number of rows with null 'smoothed_wtested_positive_14d': 21632


In [58]:
# Count non-null entries
non_null_count = merged_df["smoothed_wtested_positive_14d"].notnull().sum()
print(f"Number of rows with non-null 'smoothed_wtested_positive_14d': {non_null_count}")


Number of rows with non-null 'smoothed_wtested_positive_14d': 3994


In [59]:
p14d_df = merged_df[merged_df["smoothed_wtested_positive_14d"].notnull()].copy()

In [61]:
p14d_df.head()

,time_value,geo_value,smoothed_wspent_time_1d,smoothed_wtested_14d,smoothed_wpublic_transit_1d,smoothed_wworried_become_ill,smoothed_wvaccine_likely_govt_health,smoothed_wshop_1d,smoothed_wtested_positive_14d,smoothed_wwork_outside_home_1d,smoothed_wothers_masked,smoothed_wcli,smoothed_wcovid_vaccinated,smoothed_wvaccine_likely_friends,smoothed_wrestaurant_1d,smoothed_wvaccine_likely_politicians,smoothed_wvaccine_likely_who,smoothed_wwearing_mask,smoothed_wlarge_event_1d,FIPS State,FIPS County,State,County Name,smoothed_wcli_lagged,smoothed_wpublic_transit_1d_lagged,smoothed_wcovid_vaccinated_lagged
0,1/7/2021,01000,34.412959,12.541558,1.464798,65.771681,22.086274,52.854667,29.288429,30.487852,57.334785,3.244379,2.638200,28.469582,18.571266,9.482316,24.962112,84.311759,12.627098,01,000,Alabama,NaN,NaN,NaN,NaN
15,1/7/2021,02000,31.031930,23.187988,5.247612,54.332976,31.829591,52.948188,8.843686,31.242179,65.756589,0.421245,15.910167,29.240819,13.132258,10.050320,33.168119,82.025101,9.231545,02,000,Alaska,NaN,NaN,NaN,2.638200
19,1/7/2021,04000,33.894471,15.813525,1.670934,70.449040,30.601786,52.224064,30.302464,26.614385,64.266832,1.401254,4.715310,38.028305,18.465710,11.358013,33.428347,87.570662,9.285851,04,000,Arizona,NaN,3.244379,1.464798,15.910167
21,1/7/2021,04013,30.674532,16.170044,3.106310,68.928567,32.197435,50.983665,22.710935,24.467235,84.360238,2.082898,4.005679,34.448458,17.593056,11.658930,38.405013,92.846578,9.364862,04,013,Arizona,Maricopa,0.421245,5.247612,4.715310
23,1/7/2021,04019,30.489939,16.193151,2.943076,69.579201,36.636282,52.706816,21.912277,24.371077,87.764291,1.306140,6.249659,36.117471,13.814072,14.057577,42.917359,93.671275,6.418342,04,019,Arizona,Pima,1.401254,1.670934,4.005679


In [62]:
# Dictionary mapping feature to lag
lag_info = {
    "smoothed_wcli": 2,
    "smoothed_wpublic_transit_1d": 2,
    "smoothed_wcovid_vaccinated": 1,
    "smoothed_wvaccine_likely_friends": 14
}

# Apply the lag and create new columns
for feature, lag in lag_info.items():
    lagged_col_name = f"{feature}_lagged"
    p14d_df[lagged_col_name] = p14d_df[feature].shift(lag)

In [63]:
p14d_df.head()

,time_value,geo_value,smoothed_wspent_time_1d,smoothed_wtested_14d,smoothed_wpublic_transit_1d,smoothed_wworried_become_ill,smoothed_wvaccine_likely_govt_health,smoothed_wshop_1d,smoothed_wtested_positive_14d,smoothed_wwork_outside_home_1d,smoothed_wothers_masked,smoothed_wcli,smoothed_wcovid_vaccinated,smoothed_wvaccine_likely_friends,smoothed_wrestaurant_1d,smoothed_wvaccine_likely_politicians,smoothed_wvaccine_likely_who,smoothed_wwearing_mask,smoothed_wlarge_event_1d,FIPS State,FIPS County,State,County Name,smoothed_wcli_lagged,smoothed_wpublic_transit_1d_lagged,smoothed_wcovid_vaccinated_lagged,smoothed_wvaccine_likely_friends_lagged
0,1/7/2021,01000,34.412959,12.541558,1.464798,65.771681,22.086274,52.854667,29.288429,30.487852,57.334785,3.244379,2.638200,28.469582,18.571266,9.482316,24.962112,84.311759,12.627098,01,000,Alabama,NaN,NaN,NaN,NaN,NaN
15,1/7/2021,02000,31.031930,23.187988,5.247612,54.332976,31.829591,52.948188,8.843686,31.242179,65.756589,0.421245,15.910167,29.240819,13.132258,10.050320,33.168119,82.025101,9.231545,02,000,Alaska,NaN,NaN,NaN,2.638200,NaN
19,1/7/2021,04000,33.894471,15.813525,1.670934,70.449040,30.601786,52.224064,30.302464,26.614385,64.266832,1.401254,4.715310,38.028305,18.465710,11.358013,33.428347,87.570662,9.285851,04,000,Arizona,NaN,3.244379,1.464798,15.910167,NaN
21,1/7/2021,04013,30.674532,16.170044,3.106310,68.928567,32.197435,50.983665,22.710935,24.467235,84.360238,2.082898,4.005679,34.448458,17.593056,11.658930,38.405013,92.846578,9.364862,04,013,Arizona,Maricopa,0.421245,5.247612,4.715310,NaN
23,1/7/2021,04019,30.489939,16.193151,2.943076,69.579201,36.636282,52.706816,21.912277,24.371077,87.764291,1.306140,6.249659,36.117471,13.814072,14.057577,42.917359,93.671275,6.418342,04,019,Arizona,Pima,1.401254,1.670934,4.005679,NaN


In [64]:
row_count = len(p14d_df)
row_count

3994

In [65]:
p14d_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3994 entries, 0 to 25611
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   time_value                               3994 non-null   object 
 1   geo_value                                3994 non-null   object 
 2   smoothed_wspent_time_1d                  3873 non-null   float64
 3   smoothed_wtested_14d                     3879 non-null   float64
 4   smoothed_wpublic_transit_1d              3873 non-null   float64
 5   smoothed_wworried_become_ill             3903 non-null   float64
 6   smoothed_wvaccine_likely_govt_health     3903 non-null   float64
 7   smoothed_wshop_1d                        3873 non-null   float64
 8   smoothed_wtested_positive_14d            3994 non-null   float64
 9   smoothed_wwork_outside_home_1d           3873 non-null   float64
 10  smoothed_wothers_masked                  3899 non-nu

In [66]:
# Fill NaN values in numeric columns with their median
p14d_m_df = p14d_df.copy()  # Optional: avoid changing original

# Only apply to numeric columns
numeric_cols = p14d_m_df.select_dtypes(include=["number"]).columns

# Fill NaNs with median for each numeric column
p14d_m_df[numeric_cols] = p14d_m_df[numeric_cols].fillna(p14d_m_df[numeric_cols].median())

In [67]:
p14d_m_df.isnull().sum()

time_value                                    0
geo_value                                     0
smoothed_wspent_time_1d                       0
smoothed_wtested_14d                          0
smoothed_wpublic_transit_1d                   0
smoothed_wworried_become_ill                  0
smoothed_wvaccine_likely_govt_health          0
smoothed_wshop_1d                             0
smoothed_wtested_positive_14d                 0
smoothed_wwork_outside_home_1d                0
smoothed_wothers_masked                       0
smoothed_wcli                                 0
smoothed_wcovid_vaccinated                    0
smoothed_wvaccine_likely_friends              0
smoothed_wrestaurant_1d                       0
smoothed_wvaccine_likely_politicians          0
smoothed_wvaccine_likely_who                  0
smoothed_wwearing_mask                        0
smoothed_wlarge_event_1d                      0
FIPS State                                    0
FIPS County                             

In [68]:
p14d_m_df.to_csv("merged_fips_median_data.csv", index=False)

In [69]:
# Make a copy to preserve the original
p14d_sm_df = p14d_df.copy()

# Identify numeric columns
numeric_cols = p14d_sm_df.select_dtypes(include=["number"]).columns

# Group by 'County Name' and apply median fill for each group
p14d_sm_df[numeric_cols] = (
    p14d_sm_df
    .groupby("FIPS State")[numeric_cols]
    .transform(lambda col: col.fillna(col.median()))
)

# Step 2: Fill any remaining NaNs with the overall median of each column
p14d_sm_df[numeric_cols] = p14d_sm_df[numeric_cols].fillna(p14d_sm_df[numeric_cols].median())


time_value                                 0
geo_value                                  0
smoothed_wspent_time_1d                    0
smoothed_wtested_14d                       0
smoothed_wpublic_transit_1d                0
smoothed_wworried_become_ill               0
smoothed_wvaccine_likely_govt_health       0
smoothed_wshop_1d                          0
smoothed_wtested_positive_14d              0
smoothed_wwork_outside_home_1d             0
smoothed_wothers_masked                    0
smoothed_wcli                              0
smoothed_wcovid_vaccinated                 0
smoothed_wvaccine_likely_friends           0
smoothed_wrestaurant_1d                    0
smoothed_wvaccine_likely_politicians       0
smoothed_wvaccine_likely_who               0
smoothed_wwearing_mask                     0
smoothed_wlarge_event_1d                   0
FIPS State                                 0
FIPS County                                0
State                                      0
County Nam

In [70]:
p14d_sm_df.to_csv("merged_fips_median_byState_bytotal_data.csv", index=False)

In [71]:
# Make a copy to preserve the original
p14d_movingaverage_df = p14d_df.copy()

# Step 1: Convert time_value to datetime if not already
p14d_movingaverage_df["time_value"] = pd.to_datetime(p14d_movingaverage_df["time_value"])

# Step 2: Sort by time_value
p14d_movingaverage_df = p14d_movingaverage_df.sort_values("time_value")

# Step 3: Identify numeric columns
numeric_cols = p14d_movingaverage_df.select_dtypes(include=["number"]).columns

# Step 4: Apply rolling average across entire DataFrame
# Note: Reset index to allow proper rolling based on time
p14d_movingaverage_df = p14d_movingaverage_df.set_index("time_value")

# Apply 7-day rolling mean to all numeric columns (window in days, not rows)
p14d_movingaverage_df = p14d_movingaverage_df.copy()
p14d_movingaverage_df[numeric_cols] = (
    p14d_movingaverage_df[numeric_cols]
    .rolling(window="7D", min_periods=1)
    .mean()
)

# Reset index to restore time_value as a column
p14d_movingaverage_df = p14d_movingaverage_df.reset_index()


In [72]:
p14d_movingaverage_df.isnull().sum()

time_value                                    0
geo_value                                     0
smoothed_wspent_time_1d                       0
smoothed_wtested_14d                          0
smoothed_wpublic_transit_1d                   0
smoothed_wworried_become_ill                  0
smoothed_wvaccine_likely_govt_health          0
smoothed_wshop_1d                             0
smoothed_wtested_positive_14d                 0
smoothed_wwork_outside_home_1d                0
smoothed_wothers_masked                       0
smoothed_wcli                                 0
smoothed_wcovid_vaccinated                    0
smoothed_wvaccine_likely_friends              0
smoothed_wrestaurant_1d                       0
smoothed_wvaccine_likely_politicians          0
smoothed_wvaccine_likely_who                  0
smoothed_wwearing_mask                        0
smoothed_wlarge_event_1d                      0
FIPS State                                    0
FIPS County                             

In [73]:
p14d_movingaverage_df.to_csv("merged_fips_moving_average_data.csv", index=False)